In [111]:
# IMPORTS


import numpy as np
import pandas as pd
import av

In [112]:
# VIDEOFRAME INFO EXTRACTOR


def frametimes(video_path):

    # Open the video file
    container = av.open(video_path)

    # Get the video stream (usually index 0 for the first video stream)
    video_stream = container.streams.video[0]


    # Retrieve video info
    total_frames = video_stream.frames  # Number of frames
    duration_ts = video_stream.duration  # Total duration in time units (PTS)
    time_base = video_stream.time_base  # Time base to convert duration_ts to seconds
    duration_seconds = float(duration_ts * time_base) # Duration of the video in seconds

    # Print the results
    print(f"Total Frames: {total_frames}")
    print(f"Duration (PTS units): {duration_ts}")
    print(f"Time Base: {time_base}")
    print(f"Duration (seconds): {duration_seconds} \n")


    # Storage for frame intervals
    frame_intervals = []


    # Decode video frames
    for i, frame in enumerate(container.decode(video=0)):
        # TODO: If the video stream is corrupted, it could cause infinitely repeated decoding attempts. Might need if check in case it's unstable during testing

        # Update frame info
        start_time = float(frame.pts * time_base)
        frame_intervals.append(start_time)
    
        # if i < 5:
            # print(f"Frame {i + 1}: starttime {start_time}, PTS {frame.pts}, DTS {frame.dts}")

    end_time = duration_seconds
    frame_intervals.append(end_time)

    print("")
    print(frame_intervals)

    return total_frames, frame_intervals

In [113]:
# DATAFRAME


def getIMU(dataframe_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(dataframe_path, index_col=0)


    # Display the top and bottom rows
    print("Top rows:")
    print(df.head())

    print("\nBottom rows:")
    print(df.tail())

    return df

In [114]:
# ADD FRAME_INDEX TO DATAFRAME


def add_frame_index(df, total_frames, frame_intervals):

    # Make a list of all frame indices
    frame_labels = list(range(1, total_frames + 1))

    if "TIMESTAMP" not in df.columns:
        raise ValueError("The dataframe must contain a 'TIMESTAMP' column.")

    # Match FRAME_INDEX to TIMESTAMP
    df["FRAME_INDEX"] = pd.cut(df["TIMESTAMP"], bins=frame_intervals, labels=frame_labels, include_lowest=True)

    return df

In [115]:
# DATAFRAME LABELED FRAMES


def labeledframes(label_dataframe_path):
    # Load the CSV file into a pandas DataFrame
    df_label = pd.read_csv(label_dataframe_path)


    # Display the top and bottom rows
    print(df_label)

    return df_label

In [116]:
# MAKE A DICTIONARY FOR LABELS


# Function that stores frame indices to a label
def sort_frametolabels(df_label):

    label_dict = {}
    
    for _, row in df_label.iterrows():
        frame_indices = list(range(row["frame_start"], row["frame_end"] + 1))
        label = row["label"]


        if label in label_dict:
            label_dict[label].extend(frame_indices)
        else:
            label_dict[label] = frame_indices

    # for item, value in label_dict.items():
        # print(f"{item}:", value)

    return label_dict

In [117]:
# ADD LABEL TO DATAFRAME


# Function to assign labels
def assign_label(frame_index, label_dict):
    for label, frame_indices in label_dict.items():
        if frame_index in frame_indices:
            return label
    return None  # For values not in the label_dict


# Match LABEL to FRAME_INDEX
def match_labeltoframe(df, label_dict):

    if "FRAME_INDEX" not in df.columns:
        raise ValueError("The dataframe must contain a 'FRAME_INDEX' column.")

    df["LABEL"] = df["FRAME_INDEX"].apply(lambda frame_index: assign_label(frame_index, label_dict))

    return df

In [118]:
# Opslaan naar een CSV-bestand


# output_path = "data/test_imu_frameindex_label_data.csv"
# df.to_csv(output_path, index=False)

In [119]:
# FUNCTION TO RUN EVERYTHING


def runner(video_path, dataframe, dataframe_labeled_frames):

    total_frames, frame_intervals = frametimes(video_path)
    dataframe = add_frame_index(dataframe, total_frames, frame_intervals)
    label_dict = sort_frametolabels(dataframe_labeled_frames)
    dataframe = match_labeltoframe(dataframe, label_dict)
    dataframe.dropna(inplace=True) # Remove data outside video duration (before the first or after the last frame)

    return dataframe

In [ ]:
# Path to the video
video_path = "data/test_Raf_22-11-24_13_30.MP4"

# Path to DataFrame
dataframe_path = "data/test_raf_sample_imu.csv"
dataframe = getIMU(dataframe_path)
print("")

# Path to labeled DataFrame
label_dataframe_path = "data/test_frames_Raf_sample - Sheet1.csv"
dataframe_labeled_frames = labeledframes(label_dataframe_path)
print("")

runner(video_path, dataframe, dataframe_labeled_frames)